In [1]:
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
import spacy
from spacy.tokens import Span
import random
from spacy.util import minibatch, compounding
from spacy.training.example import Example
from PyPDF2 import PdfReader
from pptx import Presentation
from docx import Document
from collections import Counter
person_tagger = spacy.load("es_core_news_sm")

#Para detector de plagio
from sentence_transformers import SentenceTransformer, util
from difflib import SequenceMatcher

#Para web:
import requests
from bs4 import BeautifulSoup
import fnmatch

In [2]:
#.docx .pptx .pdf
def obtener_nombre_archivos(extension):
    archivos = []
    for archivo in os.listdir("./dataset"):
        if archivo.endswith(extension):
            archivos.append(archivo)
    return archivos

def leer_pdf(ruta_archivo):
    archivoLectura = open(ruta_archivo,'rb')
    documento = PdfReader(archivoLectura)
    texto_documento = ""

    for pagina in documento.pages:
        texto_documento += pagina.extract_text()

    texto_limpio = re.sub(r'[^a-zA-Z0-9.,áéíóúÁÉÍÓÚñÑ]+', ' ', texto_documento)
    return texto_limpio

def leer_pptx(filepath):
    presentation = Presentation(filepath)
    texto_documento = ""

    # Recorrer todas las diapositivas
    for slide in presentation.slides:
        # Recorrer todos los elementos de texto en la diapositiva
        for shape in slide.shapes:
            if shape.has_text_frame:
                for paragraph in shape.text_frame.paragraphs:
                    for run in paragraph.runs:
                        texto_documento += run.text
                        texto_documento += " "
                        
    texto_limpio = re.sub(r'[^a-zA-Z0-9.,áéíóúÁÉÍÓÚñÑ]+', ' ', texto_documento)
    return texto_limpio

def leer_docx(ruta_archivo):
    doc = Document(ruta_archivo)
    texto_documento = ""

    # Recorrer todos los párrafos del documento
    for paragraph in doc.paragraphs:
        texto_documento += paragraph.text
        texto_documento += " "

    texto_limpio = re.sub(r'[^a-zA-Z0-9.,áéíóúÁÉÍÓÚñÑ]+', ' ', texto_documento)
    return texto_limpio

def leer_web(link):
    soup = BeautifulSoup(link,'html.parser')
    buscarContenido = soup.find('body')
    parrafos = buscarContenido.find_all('p')
    
    listaStr = list()
    for elemento in parrafos:
        parrafosUrl = elemento.getText()
        texto_limpio = re.sub(r'[^a-zA-Z0-9.,áéíóúÁÉÍÓÚñÑ]+', ' ', parrafosUrl)
        listaStr.append(texto_limpio)
    
    texto_concatenado = ' '.join(listaStr)
    return texto_concatenado

def leer_documentos(nombre_archivos):
    archivos = []
    for archivo in nombre_archivos:
        if(archivo.endswith(".pdf")):
            archivos.append(leer_pdf("./dataset/" + archivo))
        if(archivo.endswith(".pptx")):
            archivos.append(leer_pptx("./dataset/" + archivo))
        if(archivo.endswith(".docx")):
            archivos.append(leer_docx("./dataset/" + archivo))
    return archivos

def leer_archivo_analizar(nombre_archivo):
    if(nombre_archivo.endswith(".pdf")):
            return leer_pdf(nombre_archivo)
    if(nombre_archivo.endswith(".pptx")):
            return leer_pptx(nombre_archivo)
    if(nombre_archivo.endswith(".docx")):
            return leer_docx(nombre_archivo)
    if(nombre_archivo.startswith("https://")):
            return leer_web(nombre_archivo)

def obtener_nombre_archivo(ruta):
    nombre_archivo = os.path.basename(ruta)
    return nombre_archivo

In [3]:
# Ejecución
nombres_archivos_pdf = obtener_nombre_archivos(".pdf")
nombres_archivos_docx = obtener_nombre_archivos(".docx")
nombres_archivos_doc = obtener_nombre_archivos(".doc")
nombres_archivos_pptx = obtener_nombre_archivos(".pptx")
nombre_archivos = []
nombre_archivos.extend(nombres_archivos_pdf)
nombre_archivos.extend(nombres_archivos_docx)
nombre_archivos.extend(nombres_archivos_doc)
nombre_archivos.extend(nombres_archivos_pptx)
corpus = leer_documentos(nombre_archivos)
nombre_archivo_a_analizar = "./documentoSospechoso.pdf"
documento_a_analizar = leer_archivo_analizar(nombre_archivo_a_analizar)

In [4]:
#documento_a_analizar

In [5]:
print("Nombre del Archivo Procesado: " + nombre_archivo_a_analizar[2:])

Nombre del Archivo Procesado: documentoSospechoso.pdf


In [6]:
#Entrenamiento
#AIEngineering - Custom Named Entity Recognition using Python
entrenamiento = [
    ("Alumno Pedro picazo",{"entities": [(0,6,"IDENTIFICADOR")]})
]

ner = person_tagger.get_pipe("ner")

for _, anotaciones in entrenamiento:
    for ent in anotaciones.get("entities"):
        ner.add_label(ent[2])

#disable_pipes = [pipe for pipe in person_tagger.pipe_names if pipe != 'ner']
#with person_tagger.disable_pipes(*disable_pipes):
optimizer = person_tagger.resume_training()
    
for iteration in range(5):
       random.shuffle(entrenamiento)
       losses = {}
        
       batches = minibatch(entrenamiento,size=compounding(3.8,4.0,1.001))
       for batch in batches:
            examples = []
            for text, annotation in batch:
                examples.append(Example.from_dict(person_tagger.make_doc(text), annotation))
            person_tagger.update(examples, drop=0.5, losses=losses, sgd=optimizer)


for texto, _ in entrenamiento:
    documento = person_tagger(texto)
    print('Entities', [(ent.text, ent.label_) for ent in documento.ents])

Entities [('Alumno Pedro', 'MISC')]


In [7]:
print("Posibles nombres del Autor:")
documento_a_analizar_clean = re.sub(r'\W+', ' ', documento_a_analizar)
tokens = word_tokenize(documento_a_analizar_clean)
stop_words = set(stopwords.words('spanish'))
tokens_documento = [token.lower() for token in tokens if token.lower() not in stop_words]
per_tagger = person_tagger(documento_a_analizar_clean)

def contains_word(text, word):
    return word in text

for autor in per_tagger.ents[0:12]:
    token_aux = word_tokenize(autor.text)
    if(autor.label_ == "PER" and len(token_aux) > 2):
        if(contains_word(autor.text,"Alumno")):
            index = autor.text.find("Alumno")
            nuevo_texto = autor.text[index:]
            nuevo_texto = nuevo_texto.replace("Alumno","")
            print(nuevo_texto)
        else:
            print(autor.text)

Posibles nombres del Autor:


In [8]:
def find_potential_plagiarized_sentences(documents, new_document):
    model = SentenceTransformer('paraphrase-distilroberta-base-v1')
    sentences = nltk.sent_tokenize(new_document)
    sentencesD = nltk.sent_tokenize(documents[0])

    # Embeddings de los documentos existentes
    document_embeddings = model.encode(sentencesD, convert_to_tensor=True)

    # Embeddings de las frases del nuevo documento
    new_embeddings = model.encode(sentences, convert_to_tensor=True)

    # Calcular la similitud de coseno entre los embeddings
    cos_sim_scores = util.pytorch_cos_sim(new_embeddings, document_embeddings)

    potential_plagiarized_sentences = []
    plagiarized_lines = []  # Lista para almacenar las líneas plagiadas del documento

    for i, sentence in enumerate(sentences):
        max_sim_score = max(cos_sim_scores[i])
        if max_sim_score > 0.6:  # Ajusta el umbral según sea necesario
            doc_index = cos_sim_scores[i].argmax().item()
            plagiarism_percentage = max_sim_score * 100
            potential_plagiarized_sentences.append((sentence, doc_index, i, plagiarism_percentage))

            if doc_index < len(sentencesD):
                plagiarized_lines.append(sentencesD[doc_index])  # Agregar la línea plagiada del documento
            else:
                plagiarized_lines.append("")  # Agregar una cadena vacía si el índice está fuera de rango

    return potential_plagiarized_sentences, plagiarized_lines


for index in range(len(corpus)):
    similitud = SequenceMatcher(None, documento_a_analizar, corpus[index]).ratio() * 100
    if similitud > 101:
        print("Documento analizado: " + nombre_archivos[index])
        print(f"Porcentaje de potencial plagio: {similitud:.2f}%")
        potential_plagiarized_sentences, plagiarized_lines = find_potential_plagiarized_sentences([corpus[index]], documento_a_analizar)
        for sentence, doc_index, sent_index, plagiarism_percentage in potential_plagiarized_sentences:
            print(f"Frase: '{sentence}', Ubicación: {sent_index}")
            if doc_index < len(plagiarized_lines):
                print(f"PFPDC:  {plagiarized_lines[doc_index]}")
            else:
                print("No se encontró la línea correspondiente.")
        print(" ")

In [9]:
"""
import spacy

def identify_main_topic(text):
    # Load the Spanish language model in spaCy
    nlp = spacy.load("es_core_news_sm")

    # Process the text
    doc = nlp(text)

    # Extract the main topic from the text
    main_topic = None

    for sentence in doc.sents:
        # Iterate through each sentence in the document

        # Check if the sentence contains a noun phrase
        if any(token.pos_ == "NOUN" for token in sentence):
            main_topic = sentence.text
            break

    return main_topic

# Example usage
text = "Márketing en internet y nueva economía Economía de experiencia Profesores Dr. Alejandro Prince Ing. Hernán Borré Ing. Maximiliano Bracho Alumno Gallazzi, Pablo Gabriel 143.370 2 Fecha de Presentación 10 04 2017 Cuestionario 1. Qué 3 elementos hacen resurgir con fuerza la idea de una economía de experiencia..."
topic = identify_main_topic(text)
print(topic)
"""

'\nimport spacy\n\ndef identify_main_topic(text):\n    # Load the Spanish language model in spaCy\n    nlp = spacy.load("es_core_news_sm")\n\n    # Process the text\n    doc = nlp(text)\n\n    # Extract the main topic from the text\n    main_topic = None\n\n    for sentence in doc.sents:\n        # Iterate through each sentence in the document\n\n        # Check if the sentence contains a noun phrase\n        if any(token.pos_ == "NOUN" for token in sentence):\n            main_topic = sentence.text\n            break\n\n    return main_topic\n\n# Example usage\ntext = "Márketing en internet y nueva economía Economía de experiencia Profesores Dr. Alejandro Prince Ing. Hernán Borré Ing. Maximiliano Bracho Alumno Gallazzi, Pablo Gabriel 143.370 2 Fecha de Presentación 10 04 2017 Cuestionario 1. Qué 3 elementos hacen resurgir con fuerza la idea de una economía de experiencia..."\ntopic = identify_main_topic(text)\nprint(topic)\n'

In [10]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("es_core_news_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization([documento_a_analizar])
tokensk = word_tokenize(documento_a_analizar_clean)
for token in tokens:
    if token in ["Profesor", "Profesores", "Alumno", "Alumnos", "Ing"]:
        break
    print(token)

Márketing
en
internet
y
nueva
economía
Economía
de
experiencia


In [11]:
#Investigar sobre LDA
def identificar_tema(texto):
    # Cargar el modelo en español de spaCy
    nlp = spacy.load('es_core_news_sm')

    # Procesar el texto con spaCy
    doc = nlp(texto)

    # Filtrar palabras irrelevantes y puntuaciones
    palabras = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct]
    palabras = [re.sub(r'[0-9]', '', palabra) for palabra in palabras]

    # Contar la frecuencia de las palabras
    frecuencias = Counter(palabras)

    # Obtener los 11 temas principales más frecuentes
    temas_principales = [tema for tema, frecuencia in frecuencias.most_common(11)]

    return temas_principales

texto = "El cambio climático es un problema global que requiere acciones urgentes."
temas = identificar_tema(documento_a_analizar)
print("Temas principales:")
for tema in temas:
    print(tema)

Temas principales:

personalización
cliente
experiencia
producto
economía
dimensiones
masiva
empresa
hablamos
referimos


In [13]:
"""
    Disclaimer: Este trabajo fue desarrollado en base a lo aprendido en la cursada de la materia de Procesamiento de Lenguaje Natural de la UTN frba,
    la misma fue dicatada por el profesor Mg. Ing. Hernán Borré (https://github.com/hernanborre)
    
    Fuentes:
    (2023). Programación en Python. ChatGPT
    https://chat.openai.com/
    
    codebasics. (2021). Named Entity Recognition (NER): NLP Tutorial For Beginners - 12. Youtube.
    https://www.youtube.com/watch?v=2XUhKpH0p4M&list=PL6qsQ_bxK9ZznUs0WiDGhtOr6A_EAHWKS&index=2&t=1087s&ab_channel=codebasics
    
    fpread. (2020). scrapear para detectar plagios python. Youtube.
    https://www.youtube.com/watch?v=OzgQ0QYxO20&list=PL6qsQ_bxK9ZznUs0WiDGhtOr6A_EAHWKS&index=4&ab_channel=fpred
    
    I know python (2021). Plagiarism detector using python. Youtube.
    https://www.youtube.com/watch?v=lRzC3w2NDg0&list=PL6qsQ_bxK9ZznUs0WiDGhtOr6A_EAHWKS&index=1&ab_channel=Iknowpython
    
    (2023). collections — Container datatypes. Python Org.
    https://docs.python.org/3/library/collections.html#collections.Counter
    
    (2023). spaCy.
    https://spacy.io/
"""

'\n    Disclaimer: Este trabajo fue desarrollado en base a lo aprendido en la cursada de la materia de Procesamiento de Lenguaje Natural de la UTN frba,\n    la misma fue dicatada por el profesor Mg. Ing. Hernán Borré\n    \n    Fuentes:\n    \n    (2023). Programación en Python. ChatGPT\n    https://chat.openai.com/\n    \n    codebasics. (2023). Named Entity Recognition (NER): NLP Tutorial For Beginners - 12. Youtube.\n    \n    (2023). spaCy.\n    https://spacy.io/\n    \n    \n    Sitios web: Apellido, Inicial del nombre del autor (si está disponible). (Año de publicación o actualización). Título de la página. Nombre del sitio web. URL\n    Ejemplo: González, J. (2023). Posicionamiento web en Google, factores SEO que más influyen en el posicionamiento en Google. As Posicionamiento Web. https://www.asposicionamientoweb.com/seo/factores-posicionamiento-web-seo-google/\n'